## Study Cluster Synchronization in the integration of the FHN model.

### First, a test

Based on what is shown in the paper (https://doi.org/10.48550/arXiv.2303.08668), first we test with a complete graph.

In [55]:
include("src/msf.jl")
zero_msf = msf_zero()
println("zero_msf = ", zero_msf)
println("msf = ", master_stability_function(zero_msf, 0))

zero_msf = 0.20373021436669875
msf = 

6.6612382763443796e-18


In [122]:
include("src/network_matrices_creation.jl")
include("src/cluster_synch.jl")
include("src/network_simulation.jl")

N = 10
eps = 0.05
a = 0.5
b = bmatrix(pi/2-0.1, eps)
G = test_matrix_for_cluster_synch();
eigenvalues, eigenvectors, clusters, s_matrices = s_matrix_method(G)
eigenvalues = unique(round.(eigenvalues, digits=8))[2:end]
critical_couplings = zero_msf./eigenvalues #unique(zero_msf./eigenvalues)[2:end]#
println("Critical Couplings: ", critical_couplings)
println("Eigenvalues: ", eigenvalues)
σ = critical_couplings[3]*01.035


Critical Couplings: [0.020373021436669875, 0.011984130256864633, 0.009701438779366607]
Eigenvalues: [10.0, 17.0, 21.0]


0.010040989136644437

In [123]:
include("src/network_simulation.jl")
using Random
x_0 = zeros(2*N) + 0.001.*randn(2*N)
x_0[2 .* (1:N) .- 1] = rand(N) .* 2 .* a .- a
x_0[2 .* (1:N)] = rand(N) .* 2 .* (-a + a^3 / 3) .- (-a + a^3 / 3)
prob = ODEProblem((dx, x, params, t) -> coupled_fhn_eom!(dx, x, params[1], params[2], params[3], G, b), x_0, (0.0, 5000.0), [a, eps, σ])
sol = solve(prob; dtmax=0.9, maxiters=1e8);

In [125]:
using GLMakie
include("src/cluster_synch.jl")
include("src/network_sol_analysis.jl")

colors = ["#5ec962", "#21918c", "#3b528b"]
f = Figure(size= (1000, 600))
uni_clusters = unique(clusters)
for (i, cluster) in enumerate(uni_clusters)
    if isempty(cluster)
        continue
    else
        if i .< length(uni_clusters) && !isempty(uni_clusters[i+1])
            cluster = setdiff(cluster[1], uni_clusters[i+1][1])
        else
            cluster = cluster[1]
        end
        println("Cluster $i: $cluster")

        ax = Axis(f[i, 1])
        ax.xlabel = "Time"
        ax.ylabel = "Synch Error"
        t_values, synch_error = local_synch_error(sol, cluster)
        lines!(ax, t_values, synch_error; label="Cluster $cluster", linewidth=1, color = colors[i])
        axislegend()
    end
end
display(f);

Cluster 1: [1, 2, 3]


Cluster 2: [4, 5, 6]
Cluster 3: [7, 8, 9, 10]


In [49]:
st_plot(sol, 4000, 5000;skip_every=5)

In [40]:
using GLMakie
using Trapz
using ProgressMeter
using Base.Threads

include("src/cluster_synch.jl")
include("src/network_sol_analysis.jl")

N_d = 50
N_realizations = 10
d_sweep = range(0.005, 0.025, length=N_d)
t_measurement = 500.0
t_transient = 1000.0
cluster_synch_averages = zeros(N_d, 3)
global_synch_averages = zeros(N_d)

f = Figure(size=(800, 600))
uni_clusters = unique(clusters)
@showprogress for k in 1:N_d
    σ = d_sweep[k]
    temp = zeros(N_realizations, 3)
    for realization in 1:N_realizations
        x_0 = zeros(2*N) + 0.0001 .* randn(2*N)
        prob = ODEProblem((dx, x, params, t) -> coupled_fhn_eom!(dx, x, params[1], params[2], params[3], G, b), x_0, (0.0, t_transient + t_measurement), [a, eps, σ])
        # alg = Tsit5()
        sol = solve(prob; dtmax=0.5)
        t_val, global_synch = synch_error_time_series(sol)
        global_synch_averages[k] += trapz(t_val[findfirst(t_val .> t_transient):end], global_synch[findfirst(t_val .> t_transient):end])/(t_measurement * N_realizations)
        for (i, cluster) in enumerate(uni_clusters)
            if isempty(cluster)
                continue
            else
                if i .< length(uni_clusters) && !isempty(uni_clusters[i+1])
                    cluster = setdiff(cluster[1], uni_clusters[i+1][1])
                else
                    cluster = cluster[1]
                end
                t_values, synch_error = local_synch_error(sol, cluster)
                # cluster_synch_averages[k, i] += trapz(t_values[findfirst(t_values .> t_transient):end], synch_error[findfirst(t_values .> t_transient):end])/(t_measurement * N_realizations)
                temp[realization, i] = trapz(t_values[findfirst(t_values .> t_transient):end], synch_error[findfirst(t_values .> t_transient):end])/(t_measurement * N_realizations)
            end
        end
    end
    cluster_synch_averages[k, :] = sum(temp, dims=1)
end

ax = Axis(f[1, 1])
ax.xlabel = "Coupling Strength"
ax.ylabel = "Average Synch Error"
for i in 1:3
    cluster = uni_clusters[i]
    if i .< length(uni_clusters) && !isempty(uni_clusters[i+1])
        cluster = setdiff(cluster[1], uni_clusters[i+1][1])
    else
        cluster = cluster[1]
    end
    lines!(ax, d_sweep, cluster_synch_averages[:, i]; label="Cluster $cluster", linewidth=1)
end
lines!(ax, d_sweep, global_synch_averages; label="Global", linestyle=:dash)
vlines!(ax, critical_couplings; label="Critical Couplings", linewidth=1, color = :red)
axislegend()
display(f);


Progress:   4%|█▋                                       |  ETA: 0:02:40

Progress:   6%|██▌                                      |  ETA: 0:02:36

Progress:   8%|███▎                                     |  ETA: 0:02:30

Progress:  10%|████▏                                    |  ETA: 0:02:23

Progress:  12%|████▉                                    |  ETA: 0:02:19

...

Progress:  92%|█████████████████████████████████████▊   |  ETA: 0:00:10

Progress:  94%|██████████████████████████████████████▌  |  ETA: 0:00:07

Progress:  96%|███████████████████████████████████████▍ |  ETA: 0:00:05

Progress:  98%|████████████████████████████████████████▏|  ETA: 0:00:02

Progress: 100%|█████████████████████████████████████████| Time: 0:02:00


In [54]:
include("src/network_sol_analysis.jl")
using GLMakie
using LaTeXStrings

f = Figure(size = (800, 200))
ax = Axis(f[1, 1])
ax.title = "Kuramoto Order Parameter"
ax.xlabel = "Time"
ax.ylabel = "Kuramoto Order Parameter"
t_val, kuramoto_val = kuramoto_time_series(sol, N)
lines!(ax, t_val, kuramoto_val)
f

### Now for a watts-strogatz matrix

In [31]:
include("src/network_matrices_creation.jl")
include("src/cluster_synch.jl")
include("src/network_simulation.jl")

N = 100
eps = 0.05
a = 0.5
b = bmatrix(pi/2-0.1, eps)
G = wattsstrogatzmatrix(N, 3, 0.3);
eigenvalues, eigenvectors, clusters, s_matrices = s_matrix_method(G)
eigenvalues = unique(round.(eigenvalues, digits=8))[2:end]
critical_couplings = zero_msf./eigenvalues #unique(zero_msf./eigenvalues)[2:end]#
println("Critical Couplings: ", critical_couplings)
println("Eigenvalues: ", eigenvalues)
println("clusters: ", clusters)
println("len_clusters: ", length(clusters))
σ = critical_couplings[1]*1.5

Critical Couplings: [0.235289889608469, 0.17583938066652555, 0.16413772750294423, 0.15089196914182765, 0.14352961414231719, 0.11644659458982634, 0.11086478202868306, 0.10936301116925075, 0.09615916285208445, 0.09405948488914877, 0.09185608718066297, 0.07928653447674934, 0.07721712721407267, 0.07573706152893474, 0.07061230088094644, 0.06732985975561522, 0.06413010578232939, 0.06276642235336631, 0.06077133232649208, 0.05988878006086601, 0.05712669149576276, 0.05596471203686149, 0.054424191911915026, 0.05228040369777671, 0.050985682457294075, 0.049406348143452926, 0.048138998525791975, 0.046734292781802994, 0.0457433812007647, 0.04521363266193584, 0.04486710544211667, 0.04324752715688891, 0.04249461715948622, 0.0416807496052846, 0.04124538499372167, 0.040446569274938775, 0.03964174052892116, 0.03961117502432488, 0.0390057706168598, 0.03856323263685014, 0.03674441505934977, 0.03634939587890362, 0.03604866032959124, 0.03495662584724902, 0.03469531432578865, 0.03424432251845193, 0.0338918830

0.3529348344127035

In [37]:
include("src/network_simulation.jl")
using Random
σ = critical_couplings[5]*1.01
println("σ = ", σ)
x_0 = zeros(2*N)
x_0[2 .* (1:N) .- 1] = rand(N) .* 2 .* a .- a
x_0[2 .* (1:N)] = rand(N) .* 2 .* (-a + a^3 / 3) .- (-a + a^3 / 3)
prob = ODEProblem((dx, x, params, t) -> coupled_fhn_eom!(dx, x, params[1], params[2], params[3], G, b), x_0, (0.0, 1000.0), [a, eps, σ])
sol = solve(prob; dtmax=0.9);

σ = 0.14496491028374037


In [38]:
include("src/network_sol_analysis.jl")
st_plot(sol, 150, 300;skip_every=1)